### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
<!--     for filename in filenames:
        print(os.path.join(dirname, filename)) -->

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries   
import pandas as pd
import numpy as np
import cv2 
import os
import glob

In [ ]:
#importing Images
images_path="../input/flickr30k/Images/flickr30k_images/"
images=glob.glob(images_path+"*.jpg")
len(images)
images[:5]
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure()
    img=cv2.imread(images[i])
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)

In [ ]:
#captions
caption_path = '../input/flickr30k/captions.txt'
captions = open(caption_path, 'rb').read().decode('utf-8').split('\n')
len(captions)

In [ ]:
captions[13]

In [ ]:
print(captions[:5])
print(len(captions[:5]))

In [ ]:
# seperate Image & Caption
print(f'{captions[:5][1]}')
img,caption = captions[:5][1].split(",")
print(img)

In [ ]:
pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[13]}')
img,caption = captions[13].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[131]}')
img,caption = captions[131].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[1310]}')
img,caption = captions[1310].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)